<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/base-dev/train_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from  sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras

In [3]:
!cp -r /content/gdrive/MyDrive/300cw .

In [63]:
train_df = pd.read_csv('300cw/train.csv')

In [61]:
train_df

,path,0,1,2,3,4,5,6,7,8,...,126,127,128,129,130,131,132,133,134,135
0,afw_cropped/1051618982_1.jpg,0.000000,0.177123,0.006005,0.310182,0.022260,0.447369,0.038199,0.575831,0.092254,...,0.529020,0.629942,0.664131,0.619828,0.529020,0.629942,0.477918,0.628356,0.426383,0.627077
1,afw_cropped/111076519_1.jpg,0.021191,0.168748,0.003003,0.299903,0.000000,0.437955,0.011492,0.564707,0.055466,...,0.594500,0.623317,0.709949,0.644278,0.582565,0.713150,0.509565,0.713106,0.440101,0.695184
2,afw_cropped/111076519_2.jpg,0.074471,0.067306,0.045225,0.206398,0.010331,0.345662,0.000000,0.481714,0.026521,...,0.607355,0.624743,0.694234,0.665885,0.558631,0.717986,0.486826,0.707885,0.427228,0.683976
3,afw_cropped/1130084326_1.jpg,0.023288,0.278672,0.008756,0.392404,0.000000,0.493124,0.017290,0.602401,0.047112,...,0.222893,0.640050,0.368806,0.648567,0.222893,0.640050,0.182591,0.638028,0.154362,0.635017
4,afw_cropped/1130084326_2.jpg,0.000000,0.167109,0.000699,0.307824,0.008807,0.437624,0.032465,0.554486,0.086178,...,0.743226,0.666327,0.786492,0.656598,0.743226,0.666327,0.688476,0.669080,0.633970,0.667300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,afw_cropped_aug/A315.jpg,0.968750,0.218543,0.953813,0.346713,0.937159,0.474616,0.905917,0.607677,0.847996,...,0.214157,0.667793,0.188782,0.666861,0.215602,0.695491,0.251445,0.714343,0.296449,0.711866
670,afw_cropped_aug/A324.jpg,1.000000,0.057292,0.987894,0.211139,0.972762,0.352462,0.922721,0.495114,0.857527,...,0.285755,0.658796,0.223777,0.663609,0.290428,0.706328,0.324935,0.710925,0.354519,0.707964
671,afw_cropped_aug/A245.jpg,0.997824,0.219829,1.000000,0.354810,0.989119,0.467430,0.968143,0.581527,0.922320,...,0.427328,0.596325,0.332895,0.612348,0.427328,0.596325,0.475134,0.602990,0.522397,0.596490
672,afw_cropped_aug/A128.jpg,0.958201,0.339874,0.938743,0.440561,0.902823,0.522982,0.860636,0.593553,0.800629,...,0.525370,0.732588,0.420987,0.699562,0.525370,0.732588,0.569277,0.746222,0.606669,0.743482


In [64]:
img_size = 192
no_channel = 3
batch_size = 2

In [65]:
data_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [66]:
train_data_gen = data_gen.flow_from_dataframe(dataframe=train_df, directory='./300cw/',x_col='path',
                                              y_col=[f"{i}" for i in range(136)], class_mode='raw', subset='training',
                                              batch_size=batch_size, seed=42, shuffle=True, target_size=(img_size, img_size),
                                              color_mode='rgb')

validation_data_gen = data_gen.flow_from_dataframe(dataframe=train_df, directory='./300cw/',x_col='path',
                                              y_col=[f"{i}" for i in range(136)], class_mode='raw', subset='validation',
                                              batch_size=batch_size, seed=42, shuffle=True, target_size=(img_size, img_size),
                                              color_mode='rgb')

Found 540 validated image filenames.
Found 134 validated image filenames.


In [33]:
# i = 0
# for _batch in train_data_gen:
#   if i == 1:
#     break
#   img = _batch[0]
#   lm = _batch[1]

#   plt.imshow(img[0])
#   plt.scatter(lm[0,0:136:2] * img_size, lm[0,1:136:2] * img_size, s=3, c='r')
#   i += 1

In [75]:
base = keras.applications.MobileNetV2(input_shape=(192,192,3),include_top=False,weights='imagenet')

x = base.output
x = keras.layers.GlobalAvgPool2D()(x)
x = keras.layers.Dense(192, activation='relu')(x)
x = keras.layers.Dense(136)(x)

model = keras.Model(inputs=base.input, outputs=x)

In [51]:
input = keras.layers.Input(shape=(img_size,img_size,no_channel))

x = keras.layers.Conv2D(32, (3,3), activation='relu')(input)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(16, (3,3), activation='relu')(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(32, (3,3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(64, (3,3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(128, (3,3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(256, (3,3), activation='relu')(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.BatchNormalization()(x)

# x = keras.layers.Conv2D(64, (3,3), activation='relu')(x)
# x = keras.layers.MaxPool2D()(x)
# x = keras.layers.BatchNormalization()(x)

# x = keras.layers.Conv2D(256, (3,3), activation='relu')(x)
# x = keras.layers.MaxPool2D()(x)
# x = keras.layers.BatchNormalization()(x)

x = keras.layers.GlobalAvgPool2D()(x)
x = keras.layers.Dense(192, activation='relu')(x)
x = keras.layers.Dense(136)(x)

model = keras.Model(inputs=input, outputs=x)

In [56]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 192, 192, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 96, 96, 32)   864         ['input_9[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 96, 96, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 96, 96, 32)   0           ['bn_Conv1[0][0]']         

In [78]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='mean_squared_error', metrics=['accuracy']) 

In [79]:
model.fit(train_data_gen,epochs=100,batch_size=batch_size,validation_data=validation_data_gen)

Epoch 1/100
270/270 [==============================] - 16s 31ms/step - loss: 0.0282 - accuracy: 0.2352 - val_loss: 0.0486 - val_accuracy: 0.2687
Epoch 2/100
270/270 [==============================] - 7s 27ms/step - loss: 0.0284 - accuracy: 0.2352 - val_loss: 0.5165 - val_accuracy: 0.0000e+00
Epoch 3/100
270/270 [==============================] - 7s 27ms/step - loss: 0.0282 - accuracy: 0.2352 - val_loss: 0.0696 - val_accuracy: 0.2687
Epoch 4/100
270/270 [==============================] - 7s 26ms/step - loss: 0.0278 - accuracy: 0.2352 - val_loss: 0.0614 - val_accuracy: 0.2761
Epoch 5/100
153/270 [================>.............] - ETA: 2s - loss: 0.0269 - accuracy: 0.2222

KeyboardInterrupt: ignored

In [ ]:
# class 

In [ ]:
!nvidia-smi

Sat May 28 14:47:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!ps -aux|grep python

root          29  0.0  0.0      0     0 ?        Z    11:58   0:04 [python3] <defunct>
root          30  0.0  0.3 156140 40912 ?        S    11:58   0:00 python3 /usr/local/bin/colab-fileshim.py
root          55  0.3  0.4 201604 61856 ?        Sl   11:58   0:36 /usr/bin/python3 /usr/local/bin/jupyter-notebook --ip=172.28.0.2 --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
root         298  0.0  0.0  18380  3088 ?        S    11:59   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         301  0.0  0.0  31272  9296 ?        S    11:59   0:00 python3 /opt/google/drive/drive-filter.py
root       16846  0.3  0.1 128724 17668 ?        Sl   14:24   0:04 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 46787 --host 127.0.0.1 --port 23968 --server-access-token 5a02b447b2e56c08e101160a500ccbf1b2

In [ ]:
!kill 17303